# Inspect time series based on Dat

In [ ]:
#Import required libraries
import os
import numpy as np
import pandas as pd
import data_reader.data_processing as proc
import matplotlib.pyplot as plt
import analyses.saturated_transient as sta
import h5py

## Generate a time series file (HDF5 format) to store all the data

### Data is essentially concentration of reactive species leaving the domain normalized by that in the uniform flow scenario

In [ ]:
sourcedatadir = "E:/Saturated_flow/EGUGoldschmidtdataset6"
hdf5directory = "Y:/Home/khurana/4. Publications/Restructuring"
h5file = h5py.File(os.path.join(hdf5directory, "Paper2", "Figurecodes","Temporal_analysis_full.h5"), mode = 'w')
for Reg in reginvest:
    basedata = np.load(os.path.join(sourcedatadir, Reg + "AR_0/NS-AH/NS-AH_df.npy"))
    basevelocity = np.mean(basedata[2, -1, :, :])
    for t in imposedtimeseries:
        directory = os.path.join(sourecdatadir,Reg + "AR_" + t)
        print (Reg, t)
        for j in Trial:
                if ((j == '52' and t == "5") or (j == '43' and t == "1")):
                    pass
                else:
                    basefile = os.path.join(Reg + "AR_0","NS-A"+j,"NS-A"+j+"_df.npy")
                    basedata = np.load(os.path.join(sourcedatadir, basefile))
                    baseconcs, flow, heads = sta.calcconcmasstimenew (basedata,0,-1,0,-1, 51, gvarnames, "Saturated")
                    data = np.load(os.path.join(directory, "NS-A"+j,"NS-A"+j"_df.npy")
                    concs, flow, heads = sta.calcconcmasstimenew (data,0,-1,0,-1, 51, gvarnames, "Saturated")
                    subconcs = concs[1:, -1, :]/baseconcs[-1, -1,:]
                    for g in gvarnames:
                        h5file.create_dataset(t + "/" + Reg + "/" + j + "/" + g, data=subconcs[:, gvarnames.index(g)])
h5file.close()

### Identify the time series and categorize in Dat categories from Paper 1 effort

In [ ]:
#Consider time series in terms of Dat.
#Sort all the values in Dat
data  = pd.read_csv(os.path.join(hdf5directory + "Paper1","Figurecodes", "mass_flux_sensitivity_generalized.csv"), sep="\t")
gvarnames = ["DOC", "DO", "Nitrogen", "TOC"]
finaldata = data[data['Chem'].isin (gvarnames)]
mymarklist = ["^", "o", "s", "d"]
reglist = ["Slow", "Medium", "Fast"]
colorlist = ["indianred", "g", "steelblue"]

finaldata.loc[finaldata["PeDa"] > 40, "PeDamark"] = 3
finaldata.loc[(finaldata["PeDa"] > 15) & (finaldata["PeDa"] < 40), "PeDamark"] = 2
finaldata.loc[(finaldata["PeDa"] > 1) & (finaldata["PeDa"] < 15), "PeDamark"] = 1
finaldata.loc[finaldata["PeDa"] < 1, "PeDamark"] = 0
labels = {0 : r'$Da_t < 1$',
          1 : r'$1 < Da_t < 15$',
          2 : r'$15 < Da_t < 40$',
          3 : r'$Da_t > 40$'}

subfinal = finaldata[["Trial", "Regime", "Chem", "Time_series","PeDamark"]]
subfinal['key'] = subfinal.Trial + subfinal.Regime + subfinal.Chem + subfinal.Time_series.astype(str)
colorcriteria = subfinal[["PeDamark", "key"]].to_dict('records')
colorcriteria = dict(zip(subfinal['key'], subfinal['PeDamark']))

PeDapalette = {0: "grey", 1: "orange", 2: "g", 3: "indianred"}

### Generate the time series again, but now categorizing each scenario instead of Regime/Trial/Chemical

In [ ]:
#Load already generated time series
hr = h5py.File(os.path.join(hdf5directory + "Paper2","Figurecodes","Temporal_analysis_full.h5"), mode = 'r')
#Write new time series in a new HDF5 file
hw = h5py.File(os.path.join(hdf5directory + "Paper2","Figurecodes","Temporal_analysis_full_Dat.h5", mode = 'w')
print(hr.keys())

for t in imposedtimeseries:
    n0=[]
    n1=[]
    n2=[]
    n3=[]
    for Reg in reginvest:
        if Reg == "Equal":
            r = "Medium"
        else:
            r = Reg    
        for g in gvarnames:
            for j in Trial:
                if ((j == '52' and t == "5") or (j == '43' and t == "1")):
                    pass
                else:                   
                    n = hr.get(t + "/" + Reg + "/" + j + "/" + g).value
                    if int(colorcriteria[j+r+g+t]) == 0:
                        n0.append(n)
                    elif int(colorcriteria[j+r+g+t]) == 1: 
                        n1.append(n)
                    elif int(colorcriteria[j+r+g+t]) == 2:
                        n2.append(n)
                    elif int(colorcriteria[j+r+g+t]) == 3:
                        n3.append(n)
    for Dat, k in zip([0,1,2,3],[n0, n1, n2, n3]):
        df = pd.DataFrame.from_records(k)
        #Then take average at each time point
        hw.create_dataset(t + "/Dat" + str(Dat) + "/mean/", data=df.mean())
        #Then take minimum at each time point
        hw.create_dataset(t + "/Dat" + str(Dat) + "/min/", data=df.min())
        #Then take maximum at each time point
        hw.create_dataset(t + "/Dat" + str(Dat) + "/max/", data=df.max())
hr.close()
hw.close()

## Plot the time series in terms of Dat

In [ ]:
#Characteristic to plot:
datafeature = "mean"
#Load dataset for time series in terms of Dat
hr = h5py.File(os.path.join(hdf5directory + "Paper2","Figurecodes","Temporal_analysis_full_Dat.h5"), mode = 'r')
#Load dataset for base velocity ratio values which will be plotted in the graph
basedata = np.load(os.path.join(sourcedatadir, "SlowAR_0/NS-AH/NS-AH_df.npy"))
basevelocity = np.mean(basedata[2, -1, :, :])
fig, ax = plt.subplots(3, 1, figsize = (10,8), sharey = True)
for t in imposedtimeseries:
    a = ax.flat[imposedtimeseries.index(t)]
    a.set_title("Time series: T"+ t, fontsize = 16)
    for Dat in [0,1,2,3]:
        n = hr.get(t + "/Dat" + str(Dat) + "/" + datafeature + "/").value
        print(np.shape(n))
        a.plot(n[1:], label = labels[Dat])
        #a.plot(np.convolve(n[1:], np.ones((3,))/3,mode = 'valid'), label = Dat)
    directory = os.path.join(sourcedatadir, "SlowAR_" + t)
    for j in ["37"]:
        a = ax.flat[imposedtimeseries.index(t)]
        data = np.load(os.path.join(directory,"NS-A"+j,"NS-A"+j+"_df.npy")
        velocity = np.mean(data[2, :, :, :], axis = (-1,-2))/basevelocity
        a.plot(np.abs(velocity[1:]), color= "gray")
a.legend()
hr.close()
picname = os.path.join(hdf5directory, "Paper2","Figurecodes","Temporal_analysis_full_data.png")
plt.savefig(picname, dpi = 300, bbox_inches = 'tight', pad_inches = 0.1)